In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# importing the dataset
df = pd.read_csv('float_merged_updated1.csv')

In [4]:
#df.head()

In [5]:
df.shape

(67, 17)

In [6]:
X = df[['WIND SPEED']]

# Training and Testing the model for Proerty loss and Its Prediction
y = np.array(df[['property loss', 'CASUALITIES']])
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor()
rf.fit(x_train, y_train)

RandomForestRegressor()

In [7]:
y_pred = rf.predict(x_test)

In [8]:
mse = mean_squared_error(y_test,y_pred)
mse

0.051110345429229466

In [9]:
# This is root mean square error
rmse = mse**0.5
rmse

0.2260759726933171

In [10]:
mae = mean_absolute_error(y_test,y_pred)
mae

0.19008061265676238

In [11]:
r2score = r2_score(y_test,y_pred)
r2score

0.5563582455417322

In [12]:
states = df["STATE"].unique().tolist()
states

['ANDAMAN AND NICOBAR ISLANDS',
 'ANDHRA PRADESH',
 'ASSAM',
 'BIHAR',
 'GOA',
 'GUJARAT',
 'KARNATAKA',
 'KERALA',
 'MADHYA PRADESH',
 'MAHARASHTRA',
 'MANIPUR',
 'MEGHALAYA',
 'MIZORAM',
 'NAGALAND',
 'ODISHA',
 'TAMIL NADU',
 'TRIPURA',
 'WEST BENGAL']

# Making the Application

In [13]:
def all_predictions(state, wind_speed):
    precautions = "Stay indoors and away from windows until the storm has passed. Take refuge in an interior room, closet or hallway on the lowest floor possible. Don't use electrical equipment or landline phones during the storm due to lightning risk. Avoid walking or driving through floodwaters. Clear loose objects from around your property that could cause damage in high winds. Stock up on emergency supplies like non-perishable foods, water, medications, batteries, and first aid kit."
    # Training 80% and Testing 20%
    state_df = df[df['STATE'] == state][['WIND SPEED', 'property loss', 'CASUALITIES']]
    X = state_df[['WIND SPEED']]

    # Training and Testing the model for Proerty loss and Its Prediction
    y = np.array(state_df[['property loss', 'CASUALITIES']])
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    rf = RandomForestRegressor()
    rf.fit(x_train, y_train)

    def prediction(wind_speed):
        l = rf.predict([[wind_speed]])[0].tolist()
        l.append(precautions)
        return l
    

    return prediction(wind_speed)

In [14]:
(all_predictions("WEST BENGAL", 100))

[0.6648858972399998,
 0.6858514489374997,
 "Stay indoors and away from windows until the storm has passed. Take refuge in an interior room, closet or hallway on the lowest floor possible. Don't use electrical equipment or landline phones during the storm due to lightning risk. Avoid walking or driving through floodwaters. Clear loose objects from around your property that could cause damage in high winds. Stock up on emergency supplies like non-perishable foods, water, medications, batteries, and first aid kit."]

In [15]:
%%capture
%pip install gradio -q

In [16]:
import gradio as gr

In [17]:
cyclone_predictor = gr.Interface(
    title = 'Cyclone Predictor',
    description = 'It takes the wind speed as km/h and returns the predicted property loss and casualties',
    fn = all_predictions,
    inputs = [gr.Dropdown(states, label='Select the State', value='WEST BENGAL'),
                gr.Number(label='Enter the wind speed in km/h', minimum=85, maximum=200, value=85,
                          info='Minumum wind speed can be 85 Km/h and Maximum wind speed can be 200 km/h')
            ],
    outputs = [gr.Number(label='The Predicted Property Loss'),
               gr.Number(label='The Predicted Casualties'),
               gr.TextArea(label="Precautions"),
            ],
    
            
    allow_flagging = "never"
)

cyclone_predictor.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
